# Graph measures

This script is used to calculate relevant graph metrics on full connectivity matrices.

### Graph metrics

Graph measures related to modularity are calculated. First, community structure is discovered using `community_louvain` algorithm for undirected, weighted networks with positive connections. Canonical **modularity** quality function is used throughout the search. Module size is controlled by the `louvain_gamma` resolution parameter (default 1). Search is conducted `louvain_reps` times and the division with highest score is stored as a representative for a network. Then node-level measures of centrality – **within-module degree z-score** and **participation coeffiecient** (measuring diversity of intermodular connections) are calculated for each network.

### File structure

Network metrics are stored under `<atlas_name>/unthr` atlas-specific directory in `bsc` directory. Within `unthr` directory there can be different directories for specific values of louvain gamma parameter named `gamma_<louvain_gamma_value>`. Aggregated graph measures are stored in gamma-specific direcotries as numpy arrays (e.g. `z_aggregated` for within-module degree z-score).

```
<atlas_name>
├── corrmats_aggregated.json
├── corrmats_aggregated.npy
├── unthr
│   ├── gamma_<louvain_gamma_value>
│   │   ├── m_aggregated.npy
│   │   ├── p_aggregated.npy
│   │   ├── q_aggregated.npy
│   │   └── z_aggregated.npy
...
└── roi_table_filtered.csv
```

In [ ]:
import json
from itertools import product
from os.path import join
from pathlib import Path

import numpy as np
import pandas as pd
from bct.algorithms.centrality import module_degree_zscore, participation_coef
from bct.algorithms.modularity import community_louvain
from dn_utils.path import path
from tqdm.notebook import tqdm

## Settings

In [ ]:
atlas = "combined_roi"

# Modularity
louvain_B = "negative_asym"
louvain_reps = 1_00
gamma_range = np.arange(0.5, 3.5, 0.25)

# Create output paths
path_corrmats = join(path["bsc"], "corrmats")
path_corrmats_unthr = join(path_corrmats, atlas, f"unthr")
Path(path_corrmats_unthr).mkdir(exist_ok=True, parents=True)

### Load data

In [ ]:
# Load correlation matrices and metadata
corrmats_aggregated = np.load(
    join(path_corrmats, atlas, "corrmats_aggregated.npy"))
with open(join(path_corrmats, atlas, "corrmats_aggregated.json"), "r") as f:
    meta = json.loads(f.read())

# Load subject exclusion
df_exclusion = pd.read_csv(
    join(path["nistats"], "exclusion/exclusion.csv"), index_col=0)
ok_index = df_exclusion["ok_all"]    
    
# Load ROI information
df_roi = pd.read_csv(join(path_corrmats, atlas, "roi_table_filtered.csv"))
    
n_subjects = len(meta["dim1"])
n_conditions = len(meta["dim2"])
n_perr_sign = len(meta["dim3"])
n_roi = len(df_roi)

### Calculate measures

In [ ]:
for gamma in gamma_range:
    
    print(f"Calculating graph measures for 𝛾 = {gamma}")
    
    gamma_str = str(float(gamma)).replace(".", "_")
    path_corrmats_unthr_gamma = join(path_corrmats_unthr, f"gamma_{gamma_str}")
    Path(path_corrmats_unthr_gamma).mkdir(exist_ok=True)

    m_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign, n_roi))
    q_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign))
    z_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign, n_roi))
    p_aggregated = np.zeros((n_subjects, n_conditions, n_perr_sign, n_roi))

    iters = product(range(n_subjects), range(n_conditions), range(n_perr_sign))
    for sub_idx, con_idx, perr_sign_idx in tqdm(list(iters)):

        corrmat = corrmats_aggregated[sub_idx, con_idx, perr_sign_idx]
        corrmat[np.diag_indices_from(corrmat)] = 0

        best_q = 0 
        for _ in range(louvain_reps):
            m, q = community_louvain(corrmat, gamma=gamma, B=louvain_B)
            if q > best_q:
                best_m = m
                best_q = q

        # Within-module degree z-score
        z_aggregated[sub_idx, con_idx, perr_sign_idx] = module_degree_zscore(
            W=corrmat, ci=best_m, flag=0)

        # Participation coefficient
        p_aggregated[sub_idx, con_idx, perr_sign_idx] = participation_coef(
            W=corrmat, ci=best_m, degree="undirected")

        # Store best values
        m_aggregated[sub_idx, con_idx, perr_sign_idx] = best_m
        q_aggregated[sub_idx, con_idx, perr_sign_idx] = best_q


    np.save(join(path_corrmats_unthr_gamma, f"m_aggregated.npy"), m_aggregated)
    np.save(join(path_corrmats_unthr_gamma, f"q_aggregated.npy"), q_aggregated)
    np.save(join(path_corrmats_unthr_gamma, f"z_aggregated.npy"), z_aggregated)
    np.save(join(path_corrmats_unthr_gamma, f"p_aggregated.npy"), p_aggregated)